## (Optional) Data Cleaning and Preparation

This notebook cleans and prepares the [UCI Online Retail II Data Set](https://archive.ics.uci.edu/ml/datasets/Online+Retail+II) for use with clustering algorithms. 

Please download the latest copy of the dataset from the following UCI link and place it in a directory called `data` before executing the notebook.

File Location: https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx

*Reference*:
 * Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
# create data file directory
dir_path = './data'

if not os.path.exists(dir_path):
    os.makedirs(dir_path)

Please download the latest copy of the source dataset from the UCI repo and place it in the `data` directory.

File Location: https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx

In [ ]:
%pip install openpyxl

In [ ]:
# read the raw xlsx data
df_raw = pd.read_excel('./data/online_retail_II.xlsx',
                       engine='openpyxl')

print(df_raw.shape, df_raw.columns)

df_raw.head()

### Data Preparation

Clean, fill missing data (zero since it represents no sales), and restructure data to match clustering algorithm requirements.

In [ ]:
# subset data needed for analysis and roll up to daily frequency
df_clean = df_raw[['StockCode', 'InvoiceDate', 'Quantity']]

df_clean['timestamp'] = df_clean['InvoiceDate'].dt.date

df_clean = (df_clean
            .groupby(['StockCode', 'timestamp'])['Quantity']
            .agg('sum')
            .reset_index())

print(df_clean.shape, df_clean.columns, df_clean.dtypes)

df_clean.head()

In [ ]:
df_pivot = df_clean.pivot(index='timestamp',
                          columns='StockCode',
                          values='Quantity')

print(df_pivot.shape, df_pivot.columns)

df_pivot.head()

In [ ]:
# columns with unusual stock code data
print(list(df_pivot.columns)[-75:])

In [ ]:
# drop columns with unusual stock code data
drop_cols = ['ADJUST', 'ADJUST2', 'AMAZONFEE', 'B', 'BANK CHARGES', 'C2', 'C3', 'D', 
             'DOT', 'GIFT', 'M', 'PADS', 'POST', 'S', 'SP1002', 'TEST001', 'TEST002',
             'gift_0001_10', 'gift_0001_20', 'gift_0001_30', 'gift_0001_40', 'gift_0001_50',
             'gift_0001_60', 'gift_0001_70', 'gift_0001_80', 'gift_0001_90', 'm']

df_pivot.drop(columns=drop_cols, inplace=True)

print(df_pivot.shape, df_pivot.columns)

df_pivot.head()

In [ ]:
# resample time series data and fill missing values with 0s
df_pivot.index = pd.DatetimeIndex(df_pivot.index)

df_pivot = df_pivot.resample('D').sum().fillna(0)

print(df_pivot.shape, df_pivot.columns)

df_pivot.head()

In [ ]:
# transpose data to match format neeed for further processing
df_final = df_pivot.T
df_final = df_final.reset_index()

print(df_final.shape, df_final.columns)

df_final.head()

In [ ]:
# back up data -> used for clustering and Forecast training in later notebooks
df_final.to_csv('./data/df_pivoted.csv.zip', index=None)

### End of processing